In [ ]:
# Import pandas
import pandas as pd

# Load dataset
cc_apps = pd.read_csv("datasets/cc_approvals.data", header=None)

# Inspect data
cc_apps.head()

In [ ]:
%%nose
import pandas as pd

def test_cc_apps_exists():
    assert "cc_apps" in globals(), \
        "The variable cc_apps should be defined."
        
def test_cc_apps_correctly_loaded():
    correct_cc_apps = pd.read_csv("datasets/cc_approvals.data", header=None)
    try:
        pd.testing.assert_frame_equal(cc_apps, correct_cc_apps)
    except AssertionError:
        assert False, "The variable cc_apps should contain the data as present in datasets/cc_approvals.data."

In [ ]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print('\n')

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print('\n')

# Inspect missing values in the dataset
cc_apps.tail(17)

In [ ]:
%%nose

def test_cc_apps_description_exists():
    assert "cc_apps_description" in globals(), \
        "The variable cc_apps_description should be defined."

def test_cc_apps_description_correctly_done():
    correct_cc_apps_description = cc_apps.describe()
    assert str(correct_cc_apps_description) == str(cc_apps_description), \
        "cc_apps_description should contain the output of cc_apps.describe()."
    
def test_cc_apps_info_exists():
    assert "cc_apps_info" in globals(), \
        "The variable cc_apps_info should be defined."

def test_cc_apps_info_correctly_done():
    correct_cc_apps_info = cc_apps.info()
    assert str(correct_cc_apps_info) == str(cc_apps_info), \
        "cc_apps_info should contain the output of cc_apps.info()."

In [6]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13
cc_apps = cc_apps.drop([11, 13], axis=1)

# Split into train and test sets
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

In [ ]:
%%nose


def test_columns_dropped_correctly():
    assert cc_apps.shape == (
        690,
        14,
    ), "The shape of the DataFrame isn't correct. Did you drop the two columns?"


def test_data_split_correctly():
    cc_apps_train_correct, cc_apps_test_correct = train_test_split(
        cc_apps, test_size=0.33, random_state=42
    )
    assert cc_apps_train_correct.equals(cc_apps_train) and cc_apps_test_correct.equals(
        cc_apps_test
    ), "It doesn't appear that the data splitting was done correctly."

In [8]:
# Import numpy
import numpy as np

# Replace the '?'s with NaN in the train and test sets
cc_apps_train = cc_apps_train.replace('?', np.NaN)
cc_apps_test = cc_apps_test.replace('?', np.NaN)

In [ ]:
%%nose

# def test_cc_apps_assigned():
#     assert "cc_apps" in globals(), \
#         "After the NaN replacement, it should be assigned to the same variable cc_apps only."


def test_cc_apps_correctly_replaced():
    cc_apps_fresh = pd.read_csv("datasets/cc_approvals.data", header=None)
    cc_apps_train_correct, cc_apps_test_correct = train_test_split(
        cc_apps_fresh, test_size=0.33, random_state=42
    )

    correct_cc_apps_replacement_correct_train = cc_apps_train_correct.replace(
        "?", np.NaN
    )
    correct_cc_apps_replacement_correct_test = cc_apps_test_correct.replace("?", np.NaN)
    string_cc_apps_replacement_train = cc_apps_train_correct.replace("?", "NaN")
    string_cc_apps_replacement_test = cc_apps_test_correct.replace("?", "NaN")
    #     assert cc_apps.to_string() == correct_cc_apps_replacement.to_string(), \
    #         "The code that replaces question marks with NaNs doesn't appear to be correct."
    try:
        pd.testing.assert_frame_equal(
            correct_cc_apps_replacement_correct_train, cc_apps_train
        )

        pd.testing.assert_frame_equal(
            correct_cc_apps_replacement_correct_test, cc_apps_test
        )
    except AssertionError:
        if string_cc_apps_replacement_train.equals(
            cc_apps_train
        ) or string_cc_apps_replacement_test.equals(cc_apps_test):
            assert (
                False
            ), 'It looks like the question marks were replaced by the string "NaN". Missing values should be represented by `np.nan`.'

In [ ]:
# Impute the missing values with mean imputation
cc_apps_train.fillna(cc_apps_train.mean(), inplace=True)
cc_apps_test.fillna(cc_apps_train.mean(), inplace=True)

# Count the number of NaNs in the datasets and print the counts to verify
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

In [ ]:
%%nose


def test_cc_apps_correctly_imputed():
    assert (
        cc_apps_train.isnull().to_numpy().sum() == 39
        and cc_apps_test.isnull().to_numpy().sum() == 15
    ), "There should be 39 and 15 null values in the training and test sets after your code is run, but there aren't."

In [ ]:
# Iterate over each column of cc_apps_train
for col in cc_apps_train.columns:
    # Check if the column is of object type
    if cc_apps_train[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps_train = cc_apps_train.fillna(cc_apps_train[col].value_counts().index[0])
        cc_apps_test = cc_apps_test.fillna(cc_apps_train[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

In [ ]:
%%nose


def test_cc_apps_correctly_imputed():
    assert (
        cc_apps_train.isnull().to_numpy().sum() == 0
        and cc_apps_test.isnull().to_numpy().sum() == 0
    ), "There should be 0 null values after your code is run, but there isn't."

In [14]:
# Convert the categorical features in the train and test sets independently
cc_apps_train = pd.get_dummies(cc_apps_train)
cc_apps_test = pd.get_dummies(cc_apps_test)

# Reindex the columns of the test set aligning with the train set
cc_apps_test = cc_apps_test.reindex(columns=cc_apps_train.columns, fill_value=0)

In [ ]:
%%nose


def test_label_encoding_done_correctly():
    for col in cc_apps_train.columns:
        if (
            np.issubdtype(cc_apps_train[col].dtype, np.number) != True
            and np.issubdtype(cc_apps_test[col].dtype, np.number) != True
        ):
            assert "It doesn't appear that all of the non-numeric columns were converted to numeric using fit_transform and transform."

In [16]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Segregate features and labels into separate variables
X_train, y_train = cc_apps_train.iloc[:, :-1].values, cc_apps_train.iloc[:, [-1]].values
X_test, y_test = cc_apps_test.iloc[:, :-1].values, cc_apps_test.iloc[:, [-1]].values

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [ ]:
%%nose


def test_training_range_set_correctly():
    min_value_in_rescaledX_train = np.amin(rescaledX_train)
    max_value_in_rescaledX_train = np.amax(rescaledX_train)
    assert (
        min_value_in_rescaledX_train == 0.0 and max_value_in_rescaledX_train == 1.0
    ), "Did you correctly fit and transform the `X_train` data?"


def test_xtest_created():
    assert (
        "rescaledX_test" in globals()
    ), "Did you correctly use the fitted `scaler` to transform the `X_test` data?"

In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train,y_train)

In [ ]:
%%nose


def test_logreg_defined():
    assert (
        "logreg" in globals()
    ), "Did you instantiate LogisticRegression in the logreg variable?"


def test_logreg_defined_correctly():
    logreg_correct = LogisticRegression()
    assert str(logreg_correct) == str(
        logreg
    ), "The logreg variable should be defined with LogisticRegression() only."

In [ ]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))

# Print the confusion matrix of the logreg model
confusion_matrix(y_test,y_pred)

In [ ]:
%%nose

def test_ypred_defined():
    assert "y_pred" in globals(),\
        "The variable y_pred should be defined."

def test_ypred_defined_correctly():
    correct_y_pred = logreg.predict(rescaledX_test)
    assert str(correct_y_pred) == str(y_pred),\
        "The y_pred variable should contain the predictions as made by LogisticRegression on rescaledX_test."

In [22]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

In [ ]:
%%nose


def test_tol_defined():
    assert "tol" in globals(), "The variable tol should be defined."


def test_max_iter_defined():
    assert "max_iter" in globals(), "The variable max_iter should be defined."


def test_tol_defined_correctly():
    correct_tol = [0.01, 0.001, 0.0001]
    assert (
        correct_tol == tol
    ), "It looks like the tol variable is not defined with the list of correct values."


def test_max_iter_defined_correctly():
    correct_max_iter = [100, 150, 200]
    assert (
        correct_max_iter == max_iter
    ), "It looks like the max_iter variable is not defined with a list of correct values."


def test_param_grid_defined():
    assert "param_grid" in globals(), "The variable param_grid should be defined."


def test_param_grid_defined_correctly():
    correct_param_grid = dict(tol=tol, max_iter=max_iter)
    assert str(correct_param_grid) == str(
        param_grid
    ), "It looks like the param_grid variable is not defined properly."

In [ ]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Fit grid_model to the data
grid_model_result = grid_model.fit(rescaledX_train, y_train)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

# Extract the best model and evaluate it on the test set
best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classifier: ", best_model.score(rescaledX_test,y_test))

In [ ]:
%%nose

correct_grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
correct_grid_model_result = correct_grid_model.fit(rescaledX_train, y_train)


def test_grid_model_defined():
    assert "grid_model" in globals(), "The variable grid_model should be defined."


def test_grid_model_defined_correctly():
    # correct_grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
    assert str(correct_grid_model) == str(
        grid_model
    ), "It doesn't appear that `grid_model` was defined correctly."


def test_grid_model_results_defined():
    assert (
        "grid_model_result" in globals()
    ), "The variable `grid_model_result` should be defined."


def test_grid_model_result_defined_correctly():
    #     correct_grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
    #     correct_grid_model_result = correct_grid_model.fit(rescaledX, y)
    assert str(correct_grid_model_result) == str(
        grid_model_result
    ), "It doesn't appear that `grid_model_result` was defined correctly."


def test_best_score_defined_correctly():
    #     correct_grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
    #     correct_grid_model_result = correct_grid_model.fit(rescaledX, y)
    correct_best_score = correct_grid_model_result.best_score_
    assert (
        correct_best_score == best_score
    ), "It looks like the variable `best_score` is not defined correctly."


def test_best_params_defined_correctly():
    #     correct_grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
    #     correct_grid_model_result = correct_grid_model.fit(rescaledX, y)
    correct_best_params = correct_grid_model_result.best_params_
    assert (
        correct_best_params == best_params
    ), "It looks like the variable `best_params` is not defined correctly."


def test_best_model_defined_correctly():
    correct_best_model = correct_grid_model_result.best_estimator_
    assert (
        str(correct_best_model) == str(best_model)
    ), "It looks like the variable `best_model` is not defined correctly."